In [1]:
import pandas as pd
import matplotlib.pyplot as plt  # To visualize
import pandas as pd  # To read data
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [2]:
driver = pd.read_csv("driver_ids.csv")
rider = pd.read_csv("ride_ids.csv")
timestamps = pd.read_csv("ride_timestamps.csv")

In [3]:
pivot_timestamps = timestamps.pivot(index='ride_id', columns='event', values='timestamp')
pivot_timestamps.reset_index(level=0, inplace=True)
pivot_timestamps

event,ride_id,accepted_at,arrived_at,dropped_off_at,picked_up_at,requested_at
0,00003037a262d9ee40e61b5c0718f7f0,2016-06-13 09:39:51,2016-06-13 09:44:31,2016-06-13 10:03:05,2016-06-13 09:44:33,2016-06-13 09:39:19
1,00005eae40882760d675da5effb89ae3,2016-05-14 05:23:25,2016-05-14 05:26:17,2016-05-14 05:34:17,2016-05-14 05:26:18,2016-05-14 05:23:21
2,000061d42cf29f73b591041d9a1b2973,2016-05-16 15:43:14,2016-05-16 15:47:29,2016-05-16 15:54:18,2016-05-16 15:47:32,2016-05-16 15:43:09
3,00006efeb0d5e3ccad7d921ddeee9900,2016-05-11 19:29:43,2016-05-11 19:35:11,2016-05-11 19:40:47,2016-05-11 19:35:15,2016-05-11 19:29:36
4,0000d9b24d8ccdd991b76258e616fa01,2016-04-26 18:12:15,2016-04-26 18:16:52,2016-04-26 18:24:14,2016-04-26 18:16:55,2016-04-26 18:11:38
...,...,...,...,...,...,...
194076,fffebb46effb90ce758fdff68b569355,2016-06-16 09:57:12,2016-06-16 09:58:54,2016-06-16 10:15:08,2016-06-16 09:58:57,2016-06-16 09:57:07
194077,ffff9c38173ddfa17bdd115c463e329f,2016-06-03 18:34:07,2016-06-03 18:35:20,2016-06-03 18:50:08,2016-06-03 18:35:22,2016-06-03 18:34:00
194078,ffff9e431f7fade54b2550ac19ab6e64,2016-06-12 04:00:10,2016-06-12 04:04:35,2016-06-12 04:15:42,2016-06-12 04:04:37,2016-06-12 04:00:10
194079,ffffa07f48f2f885f714765d794ad596,2016-04-19 01:09:12,2016-04-19 01:10:49,2016-04-19 01:30:48,2016-04-19 01:10:50,2016-04-19 01:08:57


In [4]:
rider['duration_min'] = rider['ride_duration'].apply(lambda x: x / 60)


In [5]:
rider['time_money'] = rider['duration_min'].apply(lambda x: x * 0.22)


In [6]:
rider['distance_mi'] = rider['ride_distance'].apply(lambda x: x *  0.00062137)

In [7]:
rider['dist_money'] = rider['distance_mi'].apply(lambda x: x * 1.15)

In [8]:
rider['total cost'] = ((rider['dist_money'] + rider['time_money']) * (1 + (0.01 * rider['ride_prime_time']))) + 2 + 1.75

In [9]:
rider['total cost'] = rider['total cost'].apply(lambda x: x if x <= 400 else 400)

In [10]:
rider['total cost'] = rider['total cost'].apply(lambda x: x if x >= 5 else 5)

# Timestamps + Costs

In [11]:
drive = rider.drop(['ride_id', 'ride_distance', 'ride_duration',], axis = 1)

In [12]:
combined_timestamps = pd.merge(rider,pivot_timestamps,on = 'ride_id', how = 'inner')
combined_timestamps = combined_timestamps.drop(['ride_distance','ride_duration'], axis = 1)
combined_timestamps['picked_up_at'] = pd.to_datetime(combined_timestamps['picked_up_at'].str.strip(), format='%Y-%m-%d %H:%M:%S')
combined_timestamps.dtype

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [ ]:
holder = pd.DataFrame({'rider_id': ['002be0ffdc997bd5c50703158b7c2491'], 'ride_prime_time': 0, 'duration_min':  'driver_id':['002be0ffdc997bd5c50703158b7c2491'],'accepted_at': ['2019-09-01 01:01:01'], 'arrived_at': ['2019-09-01 01:01:01'], 'dropped_off_at':['2019-09-01 01:01:01'], 'picked_up_at':['2019-09-01 01:01:01'], 'requested_at': ['2019-09-01 01:01:01']}) 
holder['picked_up_at'] = pd.to_datetime(holder['picked_up_at'].str.strip(), format='%Y-%m-%d %H:%M:%S')
#holder
combined_timestamps.append(holder)
#combined_timestamps.fillna(0)

In [ ]:
sample1 = combined_timestamps[combined_timestamps['driver_id'] == '002be0ffdc997bd5c50703158b7c2491']
sample2 = combined_timestamps[combined_timestamps['driver_id'] == 'ffff51a71f2f185ec5e97d59dbcd7a78']

In [ ]:
sample1.plot.line(x = 'picked_up_at', y = 'total cost')
sample1.plot.line(x = 'picked_up_at', y = 'ride_prime_time')

In [ ]:
sample2.plot.line(x = 'picked_up_at', y = 'total cost')
sample2.plot.line(x = 'picked_up_at', y = 'ride_prime_time')

In [ ]:
test = combined_timestamps
#test.set_index(['picked_up_at'],inplace=True)


In [ ]:
test = test.drop(['ride_id','ride_prime_time','duration_min','time_money','distance_mi','dist_money','accepted_at','arrived_at','dropped_off_at','requested_at'], axis = 1)


In [ ]:
test

In [ ]:
test.rename(columns={'total cost':'total_cost'}, inplace=True)
test['Time stamp'] = pd.to_datetime(test['picked_up_at'].str.strip(), format='%Y-%m-%d %H:%M:%S')

In [ ]:
test['just_date'] = test['Time stamp'].dt.date

In [ ]:
pd.set_option('display.max_rows', 1000)
test_sample = test[:277]

In [ ]:
test_sample

In [ ]:
fig,ax = plt.subplots()
for name in test_sample['driver_id'].tolist():
    ax.plot(test_sample[test_sample.driver_id=='driver_id'].just_date,test_sample[test_sample.driver_id=='driver_id'].total_cost,label=name)

ax.set_xlabel("driver")
ax.set_ylabel("money")

# Totaling everything

In [ ]:
df = pd.DataFrame()

df = drive.groupby(['driver_id'], as_index=False).size()
df = df.to_frame().reset_index()
df = df.rename(columns= {0: 'rides'})
drive = drive.groupby(['driver_id'],as_index=False).sum()
combined = pd.merge(drive,df,on = 'driver_id', how = 'inner')

In [ ]:
combined_date = pd.merge(combined,driver, on = 'driver_id', how = 'inner')

In [ ]:
combined_date['average prime'] = combined_date['ride_prime_time'] / combined_date['rides']

In [ ]:
combined_date

In [ ]:
pred = combined_date.drop(['driver_id', 'average prime', 'driver_onboard_date', 'time_money', 'dist_money'], axis = 1)

In [ ]:
pred

In [ ]:
numerical_columns = [col for col in pred.columns if (pred[col].dtype=='int64' or pred[col].dtype=='float64') and col != 'Exited']

In [ ]:
pred[numerical_columns].describe().loc[['min','max', 'mean','50%'],:]

In [ ]:
plt.hist(combined_date["ride_prime_time"],bins=20)

In [ ]:
plt.hist(combined_date["duration_min"],bins=20)

In [ ]:
plt.hist(combined_date["distance_mi"],bins=20)

In [ ]:
plt.hist(combined_date["total cost"],bins=20)

In [ ]:
feat = pred.drop(columns=['total cost'],axis=1)
label = pred["total cost"]
X_train, X_test, y_train, y_test = train_test_split(feat, label, test_size=0.3)

In [ ]:
sc_x = StandardScaler()
X_train = sc_x.fit_transform(X_train)
X_test = sc_x.fit_transform(X_test)
lab_enc = preprocessing.LabelEncoder()


In [ ]:
#training_scores_encoded = lab_enc.fit_transform(y_train)

In [ ]:
support_vector_classifier = SVC(kernel='rbf')
support_vector_classifier.fit(X_train,training_scores_encoded)
y_pred_svc = support_vector_classifier.predict(X_test)

In [ ]:
cm_support_vector_classifier = confusion_matrix(y_test,y_pred_svc)
print(cm_support_vector_classifier,end='\n\n')

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb_classifier = XGBClassifier()
xgb_classifier.fit(X_train,y_train)
y_pred_xgb = xgb_classifier.predict(X_test)

In [ ]:
cm_xgb_classifier = confusion_matrix(y_test,y_pred_xgb)
print(cm_xgb_classifier,end='\n\n')